In [1]:
import numpy as np
import torch
from torchmetrics.regression import MeanSquaredError, PearsonCorrCoef

当相关系数为正时，用 `0 - PearsonCorrCoef()` 作为损失函数会得到负值。

In [2]:
pred = torch.tensor([1, 2, 3, 4]).to(torch.float32)
target = torch.tensor([2, 3, 5, 10]).to(torch.float32)
np.corrcoef(target, pred)[0, 1]

0.9431191251430151

In [3]:
loss_fn = PearsonCorrCoef()
loss_fn(pred, target)

tensor(0.9431)

In [4]:
loss_fn = 0 - PearsonCorrCoef()
loss_fn(pred, target)

tensor(-0.9431)

当相关系数为负时，用 `0 - PearsonCorrCoef()` 作为损失函数会得到正值。

In [5]:
pred = torch.tensor([-2, 0, -5, -8]).to(torch.float32)
np.corrcoef(target, pred)[0, 1]

-0.909826216965074

In [6]:
loss_fn = PearsonCorrCoef()
loss_fn(pred, target)

tensor(-0.9098)

In [7]:
loss_fn = 0 - PearsonCorrCoef()
loss_fn(pred, target)

tensor(0.9098)

查看 `0 - PearsonCorrCoef()` 的类型，可以发现它是一个 `CompositionalMetric`。

In [8]:
type(0 - PearsonCorrCoef())

torchmetrics.metric.CompositionalMetric

注意，其中的 `0` 是必要的，也就是不能用 `-PearsonCorrCoef()`。

根据 [torchmetrics/metric.py](https://github.com/Lightning-AI/torchmetrics/blob/7c16833db55cc1d36819701fa78ada33fdda33df/src/torchmetrics/metric.py#L1164-L1166) 中的 `__neg__` 方法，`-PearsonCorrCoef()` 会返回一个 `CompositionalMetric`，其中 `operator` 是 `_neg`，其定义如下：

```python
def _neg(x: Tensor) -> Tensor:
    return -torch.abs(x)
```

如果缺少了 `0`，而只用 `-PearsonCorrCoef()`，那么得到的值是：先计算皮尔森相关系数，再取绝对值，最后取负值。因此无论如何都会得到一个非正数，这并不是我们想要的。

In [9]:
loss_fn = -PearsonCorrCoef()
loss_fn(pred, target)

tensor(-0.9098)

利用魔法方法实现多个指标之间的算子

In [10]:
loss_fn = MeanSquaredError() - 2 * PearsonCorrCoef()
loss_fn(pred, target)

tensor(114.0696)

In [11]:
MeanSquaredError()(pred, target)

tensor(112.2500)

In [12]:
PearsonCorrCoef()(pred, target)

tensor(-0.9098)